## FunTime比價網
### 爬取網路上 最便宜的四個網路訂票網站
### 提供 商家、內容、房價資訊，並限定有AsiaYo的住宿點

In [1]:
import re
import json
import requests
from bs4 import BeautifulSoup

headers = json.loads(r'''{
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7",
    "Cache-Control": "max-age=0",
    "Connection": "keep-alive",
    "Cookie": "packet_uid=0A58A13E-CDF5-9DCA-3624-33440A670283; PHPSESSID=jv9g81rkd92bbk5j4cqbfoeqn7; user=domhotel%7CTY%7C2018-05-23+21%3A42%3A49%3Bdomhotel%7CTY%7C2018-05-23+18%3A30%3A20%3Bdomhotel%7CYL%7C2018-05-23+18%3A28%3A14%3Bdomhotel%7CTY%7C2018-05-22+17%3A15%3A58%3B%3B%3B",
    "Host": "www.funtime.com.tw",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36 OPR/52.0.2871.64"
}''')

In [2]:
# 訂房地點查宜蘭住宿，雙人房
# 不限雙人房: roomtype_array[]=none
url = 'https://www.funtime.com.tw/domhotel/result.php?h_name=&checkin=2018-06-27&checkout=2018-06-28&city=YL&district=&viewspot=&sk=&roomtype_array[]=DBL&breakfast_array[]=none&price_array[]=none&roomprice_min=0&roomprice_max=30000&viewpoint_array[]=none&facility_array[]=none&source[]=none&&page=1&per_page=40&sort=hot_h&search=a'
resp = requests.get(url, headers=headers)
soup = BeautifulSoup(resp.text, 'lxml')

In [3]:
# 獲取總筆數
total_room = int(re.findall('住宿共 (\d+) 家飯店', resp.text)[0])
total_room

983

In [4]:
# 預設一頁顯示 40筆，推敲總頁數
# 第二頁網址後面是'&page=2'
total_page = int(total_room / 40)
total_page

24

In [6]:
'''
unfinished
'''

## 爬蟲使用方法
for num in list(range(1, 25)):
    url = 'https://www.funtime.com.tw/domhotel/result.php?h_name=&checkin=2018-06-27&checkout=2018-06-28&city=YL&district=&viewspot=&sk=&roomtype_array[]=DBL&breakfast_array[]=none&price_array[]=none&roomprice_min=0&roomprice_max=30000&viewpoint_array[]=none&facility_array[]=none&source[]=none&&page={}&per_page=40&sort=hot_h&search=a'.format(num)
    ## 接下來去蟲...

### 過濾所需資料

![](img/funtime1.JPG)

In [5]:
infos = soup.find_all("div", {"class":"result_center"})

In [6]:
# 商家
sources = soup.find_all("div", {"class":"price_source"})
source = [s.text.strip() for s in sources]
source[:9]

['Trip', '-', '-', 'agoda', 'Expedia', 'Booking', 'agoda', 'Booking', 'Trip']

In [7]:
# 詳細頁面連結
links = soup.find_all("a", {"class":"to_detail"})
link = ['https://www.funtime.com.tw/domhotel/' + l['href'] for l in links]
link[:9]

['https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1232518577484',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1232518577484',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1232518577484',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1232518577484',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1221811299437_81',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1221811299437_81',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1221811299437_81',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1221811299437_81',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1437542682817']

In [8]:
# 詳細頁面有四個重複
# 重新設定詳細連結
# 為了保持順序，不能用set去除重複連結
roomlink = []
for l in link[::4]:
    roomlink.append(l)
roomlink[:3]

['https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1232518577484',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1221811299437_81',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1437542682817']

### 筆記: [改變讀取順序](https://blog.csdn.net/iflysoft/article/details/9013315)

<tr>
    <td>
        <img src="img/funtime2.JPG", width=470, height=363, style="float:left;">
    </td>
</tr>

In [9]:
# 最便宜商家
stores = soup.find_all("div", {"class":"go_source"})
store = [s.text.lstrip('最低價：').strip() for s in stores]
store[:9]

['雄獅',
 '雄獅',
 'AsiaYo',
 'Hotels',
 'Booking',
 'agoda',
 'agoda',
 'Trip',
 'Booking']

### 筆記: [刪除特定字串](http://python3-cookbook-personal.readthedocs.io/zh_CN/latest/c02/p11_strip_unwanted_characters.html)

## 過濾非AsiaYo的連結

In [10]:
i = 0 # 陣列第幾個位置
asiayoLink = [] # 包含AsiaYo的連結
for s in source:
    if s == 'AsiaYo':
        asiayoLink.append(roomlink[int(i/3)]) # 3個一循環
    i += 1

In [11]:
asiayoLink

['https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1257401791015',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1349922814875_140711',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1467339129108',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1460692002437',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1494252032903',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1441874034745',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1349922808609']

In [12]:
i = 0 # 陣列第幾個位置
for s in store:
    if s == 'AsiaYo':
        asiayoLink.append(roomlink[i])
    i += 1

In [13]:
asiayoLink

['https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1257401791015',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1349922814875_140711',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1467339129108',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1460692002437',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1494252032903',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1441874034745',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1349922808609',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1437542682817',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1460692002437']

## 整理成function

In [14]:
def searchAsiaYo(soup):
    infos = soup.find_all("div", {"class":"result_center"})
    # 商家
    sources = soup.find_all("div", {"class":"price_source"})
    source = [s.text.strip() for s in sources]
    # 最便宜商家
    stores = soup.find_all("div", {"class":"go_source"})
    store = [s.text.lstrip('最低價：').strip() for s in stores]
    # 詳細頁面連結
    links = soup.find_all("a", {"class":"to_detail"})
    link = ['https://www.funtime.com.tw/domhotel/' + l['href'] for l in links]
    # 重新設定詳細連結
    roomlink = []
    for l in link[::4]:
        roomlink.append(l)
    
    # 過濾非AsiaYo的連結
    i = 0 # 陣列第幾個位置
    asiayoLink = [] # 包含AsiaYo的連結
    for s in source:
        if s == 'AsiaYo':
            asiayoLink.append(roomlink[int(i/3)])
        i += 1
    i = 0 # 陣列第幾個位置
    for s in store:
        if s == 'AsiaYo':
            asiayoLink.append(roomlink[i])
        i += 1
    
    return asiayoLink

In [15]:
searchAsiaYo(soup)

['https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1257401791015',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1349922814875_140711',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1467339129108',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1460692002437',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1494252032903',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1441874034745',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1349922808609',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1437542682817',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1460692002437']

### 旅館詳細頁面
### 爬取 商家、內容、每晚均價

In [16]:
headers = json.loads(r'''{
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7", 
    "Connection": "keep-alive",
    "Cookie": "packet_uid=0A58A13E-CDF5-9DCA-3624-33440A670283; day1remind_box_DHC3=2; PHPSESSID=spsunenkeloohmoa2vdsbd85u0; user=domhotel%7CYL%7C2018-05-24+09%3A03%3A51%3Bdomhotel%7CYL%7C2018-05-23+23%3A30%3A23%3Bdomhotel%7CYL%7C2018-05-23+22%3A27%3A42%3Bdomhotel%7CYL%7C2018-05-23+22%3A26%3A08%3Bdomhotel%7CYL%7C2018-05-23+21%3A53%3A09%3Bdomhotel%7CYL%7C2018-05-23+21%3A52%3A59%3B",
    "Host": "www.funtime.com.tw",
    "Referer": "https://www.funtime.com.tw/domhotel/%E5%AE%9C%E8%98%AD%E4%BD%8F%E5%AE%BF",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36 OPR/52.0.2871.64"
}''')

### 進入詳細頁面，帶入查詢條件

In [17]:
checkin = '2018-06-27'
checkout = '2018-06-28'
roomtype = 'DBL' #雙人房

pages = [] #把變數pages刷新
pages = [link + '&checkin={}&checkout={}&&&&&&roomtype_array%5B%5D={}&'.format(checkin, checkout, roomtype) for link in asiayoLink]

url = pages[0] #隨便挑一個測試，重點在於擷取內文頁的資訊
resp = requests.get(url, headers=headers)
soup = BeautifulSoup(resp.text, 'lxml')

In [18]:
# 商家、房型、內容、早餐...資訊
hotelinfos = soup.find_all("div", {"class":"result_table"})[0]

#### 過濾不必要的，取前四名

In [19]:
# 商家
cheapstores = hotelinfos.find_all("div", {"class":"result_source"})
cheapstore = [store.text.strip() for store in cheapstores]
cheapstores = [cheapstore[1], cheapstore[3], cheapstore[5], cheapstore[7]]
cheapstores

['agoda', 'agoda', 'Expedia', 'Booking']

In [20]:
# 內容
cheapcontents = hotelinfos.find_all("div", {"class":"result_type"})
cheapcontent = [content.text.strip() for content in cheapcontents]
cheapcontents = [cheapcontent[1], cheapcontent[3], cheapcontent[5], cheapcontent[7]]
cheapcontents

['標準房(2張單人床)(Standard Two Single Beds)',
 '標準房(2張單人床)(Standard Two Single Beds) (含早餐與晚餐)',
 '標準雙床房',
 '雙床間 - 帶陽台 - 免費取消']

In [21]:
# 定價
cheapprices = hotelinfos.find_all("div", {"class":"result_price"})
cheapprice = [price.text.strip() for price in cheapprices]
cheapprices = [cheapprice[1], cheapprice[3], cheapprice[5], cheapprice[7]]
cheapprices

['$3,211', '$3,617', '$3,639', '$3,640']

### But 當連結內非擁有四個商家會出錯!!!
### 需要更改程式碼

In [22]:
# 選了只有一個商家的飯店測試
url = 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1473671815273&checkin=2018-06-27&checkout=2018-06-28&&&&&&roomtype_array%5B%5D=DBL&'
resp = requests.get(url, headers=headers)
soup = BeautifulSoup(resp.text, 'lxml')
hotelinfos = soup.find_all("div", {"class":"result_table"})[0]

In [23]:
cheapstores = hotelinfos.find_all("div", {"class":"result_source"})
cheapstore = [store.text.strip() for store in cheapstores]
len(cheapstore)

4

In [24]:
# 選了只有兩個商家的飯店測試
url = 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1349922808609&checkin=2018-06-27&checkout=2018-06-28&&&&&&roomtype_array%5B%5D=DBL&'
resp = requests.get(url, headers=headers)
soup = BeautifulSoup(resp.text, 'lxml')
hotelinfos = soup.find_all("div", {"class":"result_table"})[0]

In [25]:
cheapstores = hotelinfos.find_all("div", {"class":"result_source"})
cheapstore = [store.text.strip() for store in cheapstores]
len(cheapstore)

6

#### 觀察當只有一個飯店時，cheapstore陣列長度為4
#### 當兩個飯店時，cheapstore陣列長度為6

## 整理成function

In [26]:
# 透過前面的function取得所有連結
url = 'https://www.funtime.com.tw/domhotel/result.php?h_name=&checkin=2018-06-27&checkout=2018-06-28&city=YL&district=&viewspot=&sk=&roomtype_array[]=DBL&breakfast_array[]=none&price_array[]=none&roomprice_min=0&roomprice_max=30000&viewpoint_array[]=none&facility_array[]=none&source[]=none&&page=1&per_page=40&sort=hot_h&search=a'
resp = requests.get(url, headers=headers)
soup = BeautifulSoup(resp.text, 'lxml')
asiayoLink = searchAsiaYo(soup)

In [31]:
asiayoLink

['https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1257401791015',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1349922814875_140711',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1467339129108',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1460692002437',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1494252032903',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1441874034745',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1349922808609',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1437542682817',
 'https://www.funtime.com.tw/domhotel/dom_detail.php?hid=domhotel_1460692002437']

In [43]:
def compareAsiaYo(checkin, checkout, roomtype, asiayoLink):
    pages = [] #把變數pages刷新
    pages = [link + '&checkin={}&checkout={}&&&&&&roomtype_array%5B%5D={}&'.format(checkin, checkout, roomtype) for link in asiayoLink]
    for page in pages:
        resp = requests.get(page, headers=headers)
        soup = BeautifulSoup(resp.text, 'lxml')
        
        # 商家、房型、內容、早餐...資訊
        hotelinfos = soup.find_all("div", {"class":"result_table"})[0]
        # 商家
        cheapstores = hotelinfos.find_all("div", {"class":"result_source"})
        cheapstore = [store.text.strip() for store in cheapstores]
        # 內容
        cheapcontents = hotelinfos.find_all("div", {"class":"result_type"})
        cheapcontent = [content.text.strip() for content in cheapcontents]
        # 定價
        cheapprices = hotelinfos.find_all("div", {"class":"result_price"})
        cheapprice = [price.text.strip() for price in cheapprices]
        
        # 使用很粗糙的方式判斷
        length = len(cheapstores)
        try:
            if length == 4:
                cheapstores = [cheapstore[1]]
                cheapcontents = [cheapcontent[1]]
                cheapprices = [cheapprice[1]]
                length = 1
            elif length == 6:
                cheapstores = [cheapstore[1], cheapstore[3]]
                cheapcontents = [cheapcontent[1], cheapcontent[3]]
                cheapprices = [cheapprice[1], cheapprice[3]]
                length = 2
            elif length == 8:
                cheapstores = [cheapstore[1], cheapstore[3], cheapstore[5]]
                cheapcontents = [cheapcontent[1], cheapcontent[3], cheapcontent[5]]
                cheapprices = [cheapprice[1], cheapprice[3], cheapprice[5]]
                length = 3
            else:
                cheapstores = [cheapstore[1], cheapstore[3], cheapstore[5], cheapstore[7]]
                cheapcontents = [cheapcontent[1], cheapcontent[3], cheapcontent[5], cheapcontent[7]]
                cheapprices = [cheapprice[1], cheapprice[3], cheapprice[5], cheapprice[7]]
                length = 4
        except:
            print(page)
        
        dic = {}
        try:
            for i in range(0, length):
                dic[cheapstores[i]] = [cheapcontents[i], cheapprices[i]]
            print(dic)
        except:
            print('ERROR!!!')
            print(cheapstores)
            print(cheapcontents)
            print(cheapprices)
        #接儲存dic在 資料庫or檔案

In [44]:
compareAsiaYo('2018-06-27', '2018-06-28', 'DBL', asiayoLink)

{'agoda': ['標準房(2張單人床)(Standard Two Single Beds) (含早餐與晚餐)', '$3,617'], 'Expedia': ['標準雙床房', '$3,639'], 'Booking': ['雙床間 - 帶陽台 - 免費取消', '$3,640']}
{'Trip': ['夏爾雙人套房 大床', '$1,909'], 'Expedia': ['高級雙人房', '$1,955'], 'AsiaYo': ['Double room(不含早餐)', '$2,090'], 'Hotels': ['高級雙人房 (免費無線上網)(可退費-可免費取消)', '$2,100']}
{'Booking': ['標准雙人間', '$3,660'], 'AsiaYo': ['古堡館 201/202/101/102 雙人套房', '$3,660']}
{'AsiaYo': ['2A', '$2,400']}
{'Booking': ['標准大號床間 - 不退款', '$2,480'], 'agoda': ['標準雙人房(Standard Double)', '$2,480'], 'AsiaYo': ['戀愛の貓（雙人套房）', '$2,480']}
{'Booking': ['雙人間A - 帶陽台 - 免費取消', '$3,600'], 'agoda': ['雙人房A - 帶陽台(Double Room with Balcony A)', '$3,600'], 'AsiaYo': ['1F 森林浴-雙人房', '$3,600']}
{'Trip': ['標準雙人床房 大床', '$4,776'], 'AsiaYo': ['安妮樓中樓雙人城堡', '$5,000']}
{'AsiaYo': ['經濟雙人房', '$1,000'], 'agoda': ['平價房(Budget Room)', '$1,024'], 'Booking': ['標准雙人間', '$1,400'], 'Trip': ['陽光雙人房 大床', '$1,758']}
{'AsiaYo': ['2A', '$2,400']}
